# Davis-Putnam with Jeroslow-Wang Heuristic (Strict NNF)

This notebook implements the Davis-Putnam algorithm using the **Jeroslow-Wang heuristic** for variable selection. 
It is adapted to work with the **Strict NNF** type definitions provided in `cnf.ts`.

In [ ]:
import { RecursiveSet, RecursiveTuple, StructuralValue } from './recursive-set';
import { Literal, Clause, CNF, NNFNegation, getComplement } from './cnf';
import { Variable } from './propositional-logic-parser';

### Helper Functions

We update the helper functions to handle `NNFNegation` classes.

In [ ]:
/**
 * Extracts the variable from the literal l.
 */
function extractVariable(l: Literal): Variable {
  if (l instanceof NNFNegation) {
    return l.get(1) as string;
  } else {
    return l;
  }
}

In [ ]:
/**
 * Returns an arbitrary element from the set S.
 */
function arb<T extends StructuralValue>(S: RecursiveSet<T>): T | null {
  for (const x of S) {
    return x;
  }
  return null;
}

### Jeroslow-Wang Heuristic

The function `selectLiteral` chooses a literal that maximizes the score:
$$ J(l) = \sum_{C \in Clauses, l \in C} 2^{-|C|} $$

We verify existence using `clause.has(literal)`, which is efficient due to the structural hashing in `RecursiveSet`.

In [ ]:
function selectLiteral(
    Clauses: CNF,
    Variables: RecursiveSet<Variable>,
    UsedVars: RecursiveSet<Variable>
): Literal {
    let maxLiteral: Literal | null = null;
    let maxScore = -Infinity;

    for (const variable of Variables) {
        if (!UsedVars.has(variable)) {
            const pos: Literal = variable;
            const neg: Literal = new NNFNegation(variable);
      
            let posScore = 0;
            let negScore = 0;
      
            for (const C of Clauses) {
                const size = C.size;
                // Structural check: does the clause contain the literal object?
                if (C.has(pos)) {
                    posScore += Math.pow(2, -size);
                }
                if (C.has(neg)) {
                    negScore += Math.pow(2, -size);
                }
            }
            if (posScore > maxScore) {
                maxScore = posScore;
                maxLiteral = pos;
            }
            if (negScore > maxScore) {
                maxScore = negScore;
                maxLiteral = neg;
            }
        }
    }
    // Fallback if no specific score found (e.g. if set is empty)
    if (maxLiteral === null) {
        for (const v of Variables) if(!UsedVars.has(v)) {
            return v;
        }
        // Should not be reached if solvable
        return arb(Variables) as Literal;
    }
    return maxLiteral;
}

### Core Logic: Reduce & Saturate

Standard unit propagation logic using structural sets.

In [ ]:
function reduce(Clauses: CNF, l: Literal): CNF {
    const lBar = getComplement(l);
    const resultArray: Clause[] = [];

    for (const clause of Clauses) {
        if (clause.has(lBar)) {
            const literals: Literal[] = [];
            for (const lit of clause) {
                const isLBar = (lit === lBar) || 
                               (typeof lit === 'object' && typeof lBar === 'object' && lit.equals(lBar));                
                if (!isLBar) {
                    literals.push(lit);
                }
            }           
            const newClause = RecursiveSet.fromArray(literals);
            newClause.freeze(); 
            resultArray.push(newClause as Clause);
        } else if (!clause.has(l)) {
            resultArray.push(clause);
        }
    }
    const unitClause = RecursiveSet.singleton(l);
    unitClause.freeze();
    resultArray.push(unitClause as Clause);

    return RecursiveSet.fromArray(resultArray) as CNF;
}

In [ ]:
function saturate(Clauses: CNF): CNF {
  let S = Clauses;
  const Used = new RecursiveSet<Clause>();
  
  while (true) {
    const Units = new RecursiveSet<Clause>();
    for (const C of S) {
      if (C.size === 1 && !Used.has(C)) {
        Units.add(C);
      }
    }
    
    if (Units.isEmpty()) {
      break;
    }
    
    const unit = arb(Units)!;
    Used.add(unit);
    const l = arb(unit)!;
    S = reduce(S, l);
  }
  return S;
}

### Solver Implementation

Uses `selectLiteral` (JW Heuristic) for branching.

In [ ]:
function solveRecursive(
  Clauses: CNF,
  Variables: RecursiveSet<Variable>,
  UsedVars: RecursiveSet<Variable>
): CNF {
  const S = saturate(Clauses);
  
  // Check for empty clause (contradiction)
  const EmptyClause = new RecursiveSet<Literal>();
  EmptyClause.freeze();
  
  if (S.has(EmptyClause)) {
    const Falsum = new RecursiveSet<Clause>();
    Falsum.add(EmptyClause);
    return Falsum as CNF;
  }

  // Check if all clauses are units (Solution Found)
  let allUnits = true;
  for (const C of S) {
    if (C.size !== 1) {
      allUnits = false;
      break;
    }
  }
  if (allUnits) {
    return S;
  }

  // Branching using JW Heuristic
  const l = selectLiteral(S, Variables, UsedVars);
  const lBar = getComplement(l);
  const p = extractVariable(l);
  
  const nextUsedVars = UsedVars.union(RecursiveSet.fromArray([p])) as RecursiveSet<Variable>;

  // Branch 1: Set l to True
  const unitL = new RecursiveSet<Clause>();
  const cL = new RecursiveSet<Literal>();
  cL.add(l);
  cL.freeze();
  unitL.add(cL);
  
  const Result1 = solveRecursive(S.union(unitL) as CNF, Variables, nextUsedVars);
  if (!Result1.has(EmptyClause)) {
    return Result1;
  }

  // Branch 2: Set lBar to True
  const unitLBar = new RecursiveSet<Clause>();
  const cLBar = new RecursiveSet<Literal>();
  cLBar.add(lBar);
  cLBar.freeze();
  unitLBar.add(cLBar);
  
  return solveRecursive(S.union(unitLBar) as CNF, Variables, nextUsedVars);
}

function solve(Clauses: CNF): CNF {
  const Variables = new RecursiveSet<Variable>();
  for (const clause of Clauses) {
    for (const lit of clause) {
      Variables.add(extractVariable(lit));
    }
  }
  const UsedVars = new RecursiveSet<Variable>();
  return solveRecursive(Clauses, Variables, UsedVars);
}

### Formatting & Output

In [ ]:
function literal_to_str(C: Clause): string {
  const val = arb(C);
  if (val === null) return "{}";
  const l = val;

  if (l instanceof NNFNegation) {
    return `${l.get(1)} ↦ False`;
  } else {
    return `${l} ↦ True`;
  }
}

function prettify(Clauses: CNF): string {
  return Clauses.toString(true);
}

function toString(S: CNF, Simplified: CNF): string {
  const EmptyClause = new RecursiveSet<Literal>();
  EmptyClause.freeze();
  
  if (Simplified.has(EmptyClause)) {
    return `${prettify(S)} is unsolvable`;
  }

  const parts: string[] = [];
  const sortedClauses = Array.from(Simplified).sort((a,b) => a.toString().localeCompare(b.toString()));
  
  for (const C of sortedClauses) {
    parts.push(literal_to_str(C));
  }
  return '{ ' + parts.join(', ') + ' }';
}

### Test Cases

We manually construct clauses to test the solver. We use a helper `neg` to create `NNFNegation` objects easily.

In [ ]:
// Helpers for manual clause construction
const neg = (v: string) => new NNFNegation(v);
const clause = (...args: Literal[]) => {
    const c = new RecursiveSet<Literal>();
    args.forEach(a => c.add(a));
    c.freeze();
    return c;
};

// Example Clauses
const c1 = clause('r', 'p', 's');
const c2 = clause('r', 's');
const c3 = clause('p', 'q', 's');
const c4 = clause(neg('p'), neg('q'));
const c5 = clause(neg('p'), 's', neg('r'));
const c6 = clause('p', neg('q'), 'r');
const c7 = clause(neg('r'), neg('s'), 'q');
const c8 = clause(neg('p'), neg('s'));
const c9 = clause('p', neg('r'), neg('q'));
const c0 = clause(neg('p'), 'r', 'q', neg('s'));

const S = new RecursiveSet<Clause>();
[c0, c1, c2, c3, c4, c5, c6, c7, c8, c9].forEach(c => S.add(c));

console.log("Solving S:");
console.log(toString(S as CNF, solve(S as CNF)));